### Imports

In [3]:
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import pickle as pkl
import networkx as nx

from tqdm import tqdm

from tensorflow.keras.optimizers import Adam, Nadam
from scripts.utils import *

from GMI_.models import GMI, LogReg
from GMI_.utils import process


### Class Definitions

In [4]:
class BaseEmbedder:
    def __init__(self, graph, embed_shape = (128,)):
        self.embed(graph)
        self.E = list(graph.edges())
        self.graph = graph
        self.embed_shape = embed_shape
    
    def embed(self, graph):
        raise NotImplementedError
    
    def get_embedding(self):
        raise NotImplementedError
        

In [54]:
class GMIEmbedding(BaseEmbedder):
    def __init__(self, embed_dim = 64, graph = None, feature_matrix = None, use_xm = False, debug = False, batch_size = 1, nb_epochs = 500, patience = 20, ortho_ = 0.1, sparse_ = 0.1, lr = 1e-3, l2_coef = 0.0, drop_prob = 0.0, sparse = True, nonlinearity = 'prelu', alpha = 0.8, beta = 1.0, gamma = 1.0, negative_num = 5, epoch_flag = 20, model_name = 'test'):

        self.embed_dim = embed_dim
        self.debug = debug
        
        # Training Params
        self.graph = graph
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.patience = patience
        self.lr = lr
        self.l2_coef = l2_coef
        self.feature_matrix = feature_matrix
        self.drop_prob = drop_prob
        self.hid_units = embed_dim
        self.sparse = sparse
        self.nonlinearity = nonlinearity
        self.use_xm = use_xm
        self.ortho_ = ortho_
        self.sparse_ = sparse_
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.negative_num = negative_num
        self.epoch_flag = epoch_flag
        self.model_name = model_name
        
        self.time_per_epoch = None
        
        if graph is not None:
            self.embed()
        else:
            self.graph = None
    
    def embed(self):

        ####
        if self.feature_matrix is None:
            feature_matrix = np.identity(len(self.graph))
        else: 
            feature_matrix = self.feature_matrix

        adj_ori = nx.to_scipy_sparse_array(graph)
        features = sp.lil_matrix(feature_matrix)
        features, _ = process.preprocess_features(features)

        nb_nodes = features.shape[0]
        ft_size = features.shape[1]
        
        adj = process.normalize_adj(adj_ori + sp.eye(adj_ori.shape[0]))

        if self.sparse:
            sp_adj = process.sparse_mx_to_torch_sparse_tensor(adj)
        else:
            adj = (adj + sp.eye(adj.shape[0])).todense()

        features = torch.FloatTensor(features[np.newaxis])
        if not self.sparse:
            adj = torch.FloatTensor(adj[np.newaxis])

        if self.feature_matrix is not None: 
            sense_features = torch.FloatTensor(self.feature_matrix)

        model = GMI(ft_size, self.hid_units, self.nonlinearity)
        optimiser = torch.optim.Adam(model.parameters(), lr = self.lr, weight_decay = self.l2_coef)
        
#         if self.use_xm:
#              model.load_state_dict(torch.load(self.model_name + '.pkl'))
        
        if torch.cuda.is_available():
            model.cuda()
            features = features.cuda()
            sp_adj = sp_adj.cuda()
            
        b_xent = nn.BCEWithLogitsLoss()
        xent = nn.CrossEntropyLoss()
        cnt_wait = 0
        best = 1e9
        best_t = 0
        
        
        adj_dense = adj_ori.toarray()
        adj_target = adj_dense + np.eye(adj_dense.shape[0])
        adj_row_avg = 1.0 / np.sum(adj_dense, axis = 1)
        adj_row_avg[np.isnan(adj_row_avg)] = 0.0
        adj_row_avg[np.isinf(adj_row_avg)] = 0.0
        adj_dense = adj_dense * 1.0
        
        for i in range(adj_ori.shape[0]):
            adj_dense[i] = adj_dense[i] * adj_row_avg[i]
        adj_ori = sp.csr_matrix(adj_dense, dtype = np.float32)
        
        start_time = time.time()
        for epoch in tqdm(range(self.nb_epochs)):
            model.train()
            optimiser.zero_grad()
            
            res = model(features, adj_ori, self.negative_num, sp_adj, None, None) 
            
            if self.use_xm == True and feature_matrix is not None:
                
                start_idx = 0
                loop = True
                
                ortho_loss = 0
                sparse_loss = 0
                xm_batch_size = 128
                
                sf = sense_features
                embeds = model.embed(features, sp_adj)
                                
                while loop:
                    end_idx = start_idx + xm_batch_size
                    if end_idx > len(self.graph):
                        loop = False
                        end_idx = len(self.graph)
                                            
                    sf = sense_features[start_idx : end_idx]
                    embeds_ = torch.squeeze(embeds)[start_idx : end_idx]
                    
                    
                    sense_mat = torch.einsum('ij, ik -> ijk', embeds_, sf)
                    E = sense_mat
                    y_norm = torch.diagonal(torch.matmul(embeds_, torch.transpose(embeds_, 0, 1)))
                    sense_norm = torch.diagonal(torch.matmul(sf, torch.transpose(sf, 0, 1)))
                    norm = torch.multiply(y_norm, sense_norm)
                    E = torch.transpose(torch.transpose(E, 0, 2) / norm, 0, 2)
                    E = (E - torch.amin(E, dim = [-1, -2], keepdim = True)) / (torch.amax(E, dim = [-1, -2], keepdim = True) - torch.amin(E, dim = [-1, -2], keepdim = True))
                    
                    E_t = torch.transpose(E, 1, 2)
                    
                    E_o = torch.einsum('aij, ajh -> aih', E, E_t)
                    E_o = torch.sum(E_o)
                    batch_ortho_loss = (self.ortho_ * E_o) / self.batch_size

                    batch_sparse_loss = (self.sparse_ * torch.sum(torch.linalg.norm(E, ord = 1, axis = 0))) / self.batch_size
                        
                    ortho_loss += batch_ortho_loss
                    sparse_loss += batch_sparse_loss
                    
                    start_idx = end_idx
                    
                loss = self.alpha * process.mi_loss_jsd(res[0], res[1]) +\
                       self.beta * process.mi_loss_jsd(res[2], res[3]) +\
                       self.gamma * process.reconstruct_loss(res[4], adj_target) +\
                       ortho_loss +\
                       sparse_loss
            else:
                loss = self.alpha * process.mi_loss_jsd(res[0], res[1]) +\
                       self.beta * process.mi_loss_jsd(res[2], res[3]) +\
                       self.gamma * process.reconstruct_loss(res[4], adj_target)


            if self.debug:
                print('Loss:', loss)

            if loss < best:
                best = loss
                best_t = epoch
                cnt_wait = 0
                torch.save(model.state_dict(), self.model_name + '.pkl')
            else:
                cnt_wait += 1

            if cnt_wait == self.epoch_flag:
                print('Early stopping!')
                break

            loss.backward()
            optimiser.step()
            
        self.time_per_epoch = (time.time() - start_time) / epoch

        if self.debug: 
            print('Loading {}th epoch'.format(best_t))
            
        model.load_state_dict(torch.load(self.model_name + '.pkl'))

        embeds = model.embed(features, sp_adj)
        self.embeddings = embeds
    
    def get_embedding(self):
        return np.squeeze(self.embeddings.numpy())
    

In [61]:
with open('./data/usa_airport.pkl', 'rb') as file: 
    graph_dict = pkl.load(file)
    
graph = nx.Graph(nx.to_numpy_array(graph_dict['graph']))    
graph = nx.Graph(nx.to_numpy_array(graph))


sense_feat_dict, sense_features = get_sense_features(graph, ppr_flag = 'std')


1186it [00:23, 50.57it/s]


/Users/zohairshafi/miniforge3/lib/python3.10/site-packages/networkx/algorithms/centrality/katz.py:325: FutureWarning:

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.



In [62]:
uncorrelated_feats = ['Degree',
                    'Clustering Coefficient',
                    'Personalized Page Rank - Standard Deviation',
                    'Average Neighbor Degree',
                    'Average Neighbor Clustering',
                    'Eccentricity',
                    'Katz Centrality']
sense_features = sense_features[:, [list(sense_feat_dict).index(feat) for feat in uncorrelated_feats]]
sense_feat_dict = {feat : idx for idx, feat in enumerate(uncorrelated_feats)}

In [63]:
gmi_og = GMIEmbedding(graph = graph, 
           embed_dim = 128, 
           feature_matrix = sense_features, 
           use_xm = False, 
           ortho_ = 0, 
           sparse_ = 0, 
           batch_size = 1)
embed_og = gmi_og.get_embedding()
embed_og = (embed_og - np.min(embed_og)) / np.ptp(embed_og)
feature_dict_og = find_feature_membership(input_embed = embed_og,
                                                    embed_name = 'GMI-SF',
                                                    sense_features = sense_features,
                                                    sense_feat_dict = sense_feat_dict,
                                                    top_k = 8,
                                                    solver = 'nmf')

explain_og = feature_dict_og['explain_norm']
error_og = sense_features * np.log((sense_features + 1e-10) / ((embed_og @ feature_dict_og['explain_norm']) + 1e-10)) - sense_features + (embed_og @ feature_dict_og['explain_norm'])
explain_og = (explain_og - np.min(explain_og)) / np.ptp(explain_og)


gmi_plus = GMIEmbedding(graph = graph, 
           embed_dim = 128, 
           feature_matrix = sense_features, 
           use_xm = True, 
           alpha = 0.04, 
           beta = 0.5, 
           gamma = 0.5,
           ortho_ = 1e-2, 
           sparse_ = 1e-1, 
           batch_size = 1, 
           epoch_flag = 500)
embed_plus = gmi_plus.get_embedding()
embed_plus = (embed_plus - np.min(embed_plus)) / np.ptp(embed_plus)
feature_dict_plus = find_feature_membership(input_embed = embed_plus,
                                                    embed_name = 'GMI+XM',
                                                    sense_features = sense_features,
                                                    sense_feat_dict = sense_feat_dict,
                                                    top_k = 8,
                                                    solver = 'nmf')

explain_plus = feature_dict_plus['explain_norm']
error_plus = sense_features * np.log((sense_features + 1e-10) / ((embed_plus @ feature_dict_plus['explain_norm']) + 1e-10)) - sense_features + (embed_plus @ feature_dict_plus['explain_norm'])
explain_plus = (explain_plus - np.min(explain_plus)) / np.ptp(explain_plus)

100%|█████████████████████████████████████████| 500/500 [01:22<00:00,  6.04it/s]


In [64]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z = explain_og,
                         x = list(sense_feat_dict), 
                         ))
fig.update_layout(title_text = 'DGI : ' + str(np.linalg.norm(explain_og, ord = 'nuc')), 
                  xaxis_title_text = 'Sense Features', 
                  yaxis_title_text = 'Dimensions')

fig.show()

fig = go.Figure()
fig.add_trace(go.Heatmap(z = explain_plus,
                         x = list(sense_feat_dict), 
                         ))
fig.update_layout(title_text = 'DGI+ : ' + str(np.linalg.norm(explain_plus, ord = 'nuc')), 
                  xaxis_title_text = 'Sense Features', 
                  yaxis_title_text = 'Dimensions')

fig.show()

In [65]:
Y_plus = embed_plus
sense_mat = tf.einsum('ij, ik -> ijk', Y_plus, sense_features)
Y_plus_norm = tf.linalg.diag_part(tf.matmul(Y_plus, Y_plus, transpose_b = True), k = 0)
sense_norm = tf.linalg.diag_part(tf.matmul(sense_features, sense_features, transpose_b = True), k = 0)
norm = Y_plus_norm * tf.cast(sense_norm, tf.float32)
D_plus = tf.transpose(tf.transpose(sense_mat) / norm)
D_plus = (D_plus - tf.reshape(tf.reduce_min(D_plus, axis = [-1, -2]), (-1, 1, 1))) / tf.reshape(tf.reduce_max(D_plus, axis = [-1, -2]) - tf.reduce_min(D_plus, axis = [-1, -2]), (-1, 1, 1))


Y_og = embed_og

sense_mat = tf.einsum('ij, ik -> ijk', Y_og, sense_features)
Y_og_norm = tf.linalg.diag_part(tf.matmul(Y_og, Y_og, transpose_b = True), k = 0)
sense_norm = tf.linalg.diag_part(tf.matmul(sense_features, sense_features, transpose_b = True), k = 0)
norm = Y_og_norm * tf.cast(sense_norm, tf.float32)
D_og = tf.transpose(tf.transpose(sense_mat) / norm)
D_og = (D_og - tf.reshape(tf.reduce_min(D_og, axis = [-1, -2]), (-1, 1, 1))) / tf.reshape(tf.reduce_max(D_og, axis = [-1, -2]) - tf.reduce_min(D_og, axis = [-1, -2]), (-1, 1, 1))


In [66]:
norm_og = [np.linalg.norm(D_og[node, :, :], ord = 'nuc') for node in tqdm(range(len(graph)))]
norm_plus = [np.linalg.norm(D_plus[node, :, :], ord = 'nuc') for node in tqdm(range(len(graph)))]



100%|██████████████████████████████████████| 1186/1186 [00:09<00:00, 128.44it/s]


In [67]:
diff = np.array(norm_og) - np.array(norm_plus)
fig = go.Figure()
fig.add_trace(go.Histogram(x = diff))
fig.update_layout(title_text = 'Distribution of Difference In Nuclear Norms - GMI vs GMI+XM', 
                  xaxis_title_text = 'Difference In Nuclear Norms - GMI vs GMI+XM', 
                  yaxis_title_text = 'Frequency', 
                  plot_bgcolor = 'white', 
                  paper_bgcolor = 'white', 
                  font = dict(size = 30))
fig.show()

In [68]:
fig = go.Figure()

fig.add_trace(go.Histogram(x = norm_og, 
                           name = 'GMI'))
fig.add_trace(go.Histogram(x = norm_plus, 
                           name = 'GMI+XM'))


fig.update_layout(title_text = 'Distribution of Nuclear Norm of Node Explain Matrix', 
                  xaxis_title_text = 'Nuclear Norm', 
                  yaxis_title_text = 'Frequency', 
                  paper_bgcolor = 'white', 
                  plot_bgcolor = 'white', 
                  font = dict(size = 20))
fig.show()